In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#ee.Authenticate()
# geemap.update_package()

In [3]:
#list of plots in GA LTER dataset (not including broad creek)

ee.Initialize()

ga_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]), {'site_id': '1'}),
  ee.Feature(ee.Geometry.Point([-81.295564, 31.535967]), {'site_id': '2'}),
  ee.Feature(ee.Geometry.Point([-81.229141, 31.518847]), {'site_id': '3'}),
  ee.Feature(ee.Geometry.Point([-81.365805, 31.451791]), {'site_id': '4'}),
  ee.Feature(ee.Geometry.Point([-81.340722, 31.435814]), {'site_id': '5'}),
  ee.Feature(ee.Geometry.Point([-81.280011, 31.387962]), {'site_id': '6'}),
  ee.Feature(ee.Geometry.Point([-81.478498, 31.334813]), {'site_id': '7'}),
  ee.Feature(ee.Geometry.Point([-81.415535, 31.30824]), {'site_id': '8'}),
  ee.Feature(ee.Geometry.Point([-81.333707, 31.351518]), {'site_id': '9'}),
  ee.Feature(ee.Geometry.Point([-81.270863, 31.476948]), {'site_id': '10'})
]) ##Need to specify CRS? Not given, but assume NAD83 (crs ='EPSG:4269') or WGS 84?

##These are average lat/longs of the site (see GIS file); as such, need zonal spectral data (not point data)
##Lat/longs calculated in GA Biomass dataset Jupyter script

##PC path:
ga_allplots = geemap.csv_to_pandas('C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots.csv') 
##plotcoords.csv is old

ga_allplots['Date'] = pd.to_datetime(ga_allplots[['Year', 'Month', 'Day']])
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots.csv' ##This will be what you load
ga_allplots.head()

##for GA, lats and longs may differ slightly between years (compare average lat/long to recorded lat/longs in each yr)


,Year,Day,Site,Zone,Plot,Latitude,Longitude,Month,Date
0,2000,10,4,1,1,31.451676,-81.365498,10,2000-10-10
1,2000,10,4,1,2,31.451548,-81.365560,10,2000-10-10
2,2000,10,4,1,3,31.451473,-81.365566,10,2000-10-10
3,2000,10,4,1,4,31.451373,-81.365579,10,2000-10-10
4,2000,10,4,1,5,31.451197,-81.366132,10,2000-10-10


In [4]:
##landsat 5 surface reflectance images - GA LTER

gaCollectionLS5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2000, 2012,'year')) \
    .filter(ee.Filter.calendarRange(9, 11,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(gaCollectionLS5.size().getInfo())
gaimage0 = gaCollectionLS5.first()
props = geemap.image_props(gaimage0)
#props.getInfo()

100


In [5]:
##landsat 8 surface reflectance images - GA LTER
gaCollectionLS8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2018, 2018,'year')) \
    .filter(ee.Filter.calendarRange(9, 11,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

l8_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B5', 'B4', 'B3'], 
             'gamma': 1.5} ##be very careful with analysis of l8 vs l5 data - different band numbers!

print(gaCollectionLS8.size().getInfo())

14


In [6]:
#landsat 7 surface reflectance images - GA LTER - might not be able to use due to striping
gaCollectionLS7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2012, 2012,'year')) \
    .filter(ee.Filter.calendarRange(9, 11,'month')) \
    .filterBounds(ga_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

In [7]:
##get list of image ids in filtered collection for L5, L7, and L8
# gaCollectionLS5.aggregate_array('system:id').getInfo()
# gaCollectionLS8.aggregate_array('system:id').getInfo()
#gaCollectionLS7.aggregate_array('system:id').getInfo()

In [8]:
##Landsat Imagery
##Landsat times are in GMT
##Tide requirements: needs to be below 1.054 m at FORT PULASKI station (near Savannah) ID #8670870
##MHW = 2.173 m; MLW = 0.065 m
##In the future: Just use cloudless images and FLATS model to identify cloudless pictures

##landsat 5
ga00 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20001024') ##TIDE = 0.2m ##some noise here; no spots near points though
##2000 sample dates: Oct 10-13,16-18

ga01 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20011027') ##TIDE = 0.5m
##2001 sample dates: Oct 5,8,9,15,16,17 

#ga02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20021007') ##TIDE = 2.1m XXXXXX, may be best?
ga02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20020921') ##TIDE = 1.6m XXXXXX, SEPTEMBER, clouds, still best choice
##2002 sample dates: Oct 14-20

#ga03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20030915') ##TIDE = 2.2m XXXXXX ##SEPTEMBER
ga03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20031102') ##TIDE = 0.9 m ##NOVEMBER
##2003 sample dates: NONE - no data for 2003

ga04 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20041104') ##TIDE = 1.77m XXXXXX ##NOVEMBER, still the best option
##2004 sample dates: Oct 22-25

ga05 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20051015') ##TIDE = 0.74m
##2005 sample dates: Oct 21-24,26,27

ga06 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20061002') ##TIDE = 0.74m
##2006 sample dates: Oct 14-17

ga07 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20070910') ##TIDE = 0.85m ##SEPTEMBER
##2007 sample dates: Oct 20-24

#ga08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20081030') ##TIDE = 2.1m XXXXXX
ga08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20080928') ##TIDE = 1.2m XXX, SEPTEMBER, still best option
##2008 sample dates: Oct 25-28

ga09 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20091001') ##TIDE = 0.58m
##2009 sample dates: Oct 14-17

#ga10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20101029') ##TIDE = 1.93m XXXXXX
ga10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20101105') ##TIDE = 1.02m ##NOVEMBER
##2010 sample dates: Oct 18-21

#ga11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_016038_20111016') ##TIDE = 2.26m XXXXXX
ga11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_017038_20111023') ##TIDE = 0.1m
##2011 sample dates: Oct 24-27

##landsat 7
ga12 = ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_016038_20120924') ##TIDE = 0.74m, L7 XXXXXX SEPTEMBER
##2012 sample dates: Oct 12-15

##landsat 8
#ga13 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20131012') ##TIDE = 1.7m XXXXXX
ga13 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20131113') ##TIDE = 0.56m ##NOVEMBER
##2013 sample dates: Oct 13-16,18

ga14xxx = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20141008') ##TIDE = 1.25m XXXXXX
ga14 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20140922') ##TIDE = 0.79m
##2014 sample dates: Oct 15-18,20,21

ga15xxx = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20151018') ##TIDE = 2.4m XXXXXX
ga15 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20150909') ##TIDE = 0.11 ##CLOUDS
##2015 sample dates: Oct 15,20-25

#ga16 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20161029') ##TIDE = 1.09m XXXXXX; May be ok? Check flooded pixels
ga16 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20160911') ##TIDE = 0.72 ##SEPTEMBER, check sample date
##2016 sample dates: Oct 23-27

ga17 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20171016') ##TIDE = 0.28m
##2017 sample dates: Oct 14,16,18-20,23

ga18 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_016038_20181003') ##TIDE = 1.15m XXXXXX; May be ok? Check flooded pixels
##2018 sample dates: Oct 12-17,19
#ga18 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20180908') ##TIDE = 0.43m ##so many clouds, however; might be \
                                                            ## to use higher tide image? And calculate flooded pixels

ga19 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_017038_20190927') ##TIDE = 0.69m ##SEPTEMBER
##2019 sample dates: Oct 4,5,7-9,11

##determining cloud cover
cloudiness = ga00.get('CLOUD_COVER')
#print('CLOUD_COVER: ', cloudiness.getInfo())

#Map.addLayer(ga15, l8_param, "l82015")
#Map.addLayer(ga12, vis_param, "l72012")

props = geemap.image_props(ga00)
props.getInfo()

##Need flooded pixel analysis (either TMII or FLATS for Landsat)

{'CLOUD_COVER': 5,
 'CLOUD_COVER_LAND': 5,
 'EARTH_SUN_DISTANCE': 0.994431,
 'ESPA_VERSION': '2_19_0c',
 'GEOMETRIC_RMSE_MODEL': 3.781,
 'GEOMETRIC_RMSE_MODEL_X': 2.359,
 'GEOMETRIC_RMSE_MODEL_Y': 2.955,
 'IMAGE_DATE': '2000-10-24',
 'IMAGE_QUALITY': 9,
 'LANDSAT_ID': 'LT05_L1TP_017038_20001024_20160918_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1474200080000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.5.0',
 'SATELLITE': 'LANDSAT_5',
 'SENSING_TIME': '2000-10-24T15:39:31.2120380Z',
 'SOLAR_AZIMUTH_ANGLE': 149.348083,
 'SOLAR_ZENITH_ANGLE': 49.074329,
 'SR_APP_VERSION': 'LEDAPS_3.2.0',
 'WRS_PATH': 17,
 'WRS_ROW': 38,
 'system:asset_size': '342.521984 MB',
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'sr_atmos_opacity',
  'sr_cloud_qa',
  'pixel_qa',
  'radsat_qa'],
 'system:id': 'LANDSAT/LT05/C01/T1_SR/LT05_017038_20001024',
 'system:index': 'LT05_017038_20001024',
 'system:time_end': '2000-10-24 15:39:31',
 'system:time_start': '2000-10-

In [9]:
##Mapping
##Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(ga01, vis_param, "Landsat5SR")
#Map.addLayer(ga13, l8_param, "Landsat8SR")

Map.centerObject(ga_pts, 10)
#Map.set_plot_options(add_marker_cluster = True)

##add feature collection points for each plot
Map.addLayer(ga_pts, {}, "avg_plots")

Map
##what is with all the spots on ga00? Something wrong with image, salt & pepper noise?

Map(center=[31.434120927658846, -81.34324597505625], controls=(WidgetControl(options=['position', 'transparent…

<IPython.core.display.Javascript object>

In [10]:
##Adding every plot coordinate, not just averages
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots.csv'

##^^Might still be useful, use 'fc = ' function below
##Map.add_xy_data(allplots_fc, x="Longitude", y="Latitude", layer_name="csv to ee", to_ee=True)

##Adding average coordinates - change from above (allplots_fc), which is every single plot
allplots_av = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/allplots_av.csv'
allplots_av2 = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/Point_elevation/av_points.csv'
##allplots_av2 only includes Spartina plots, I believe - from GA_average data work

fc = geemap.csv_to_ee(allplots_av, latitude = "Latitude", longitude = "Longitude")

fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

fc_av = geemap.csv_to_ee(allplots_av2, latitude = "Latitude", longitude = "Longitude") ##Similar to fc, but only Spartina plots

buffplots_shp = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/Point_elevation/allplots_av_buff.shp'

buffplots = countries = geemap.shp_to_ee(buffplots_shp)

Map.addLayer(fc, {}, "fc") 
##with average plots, some variability in location. With all plots, many different (overlapping) locations

Map.addLayer(fc_all, {}, "fc_all")

Map.addLayer(fc_av, {}, "fc_av")

Map.addLayer(buffplots, {}, 'buffplots')

# in_csv = 'C:/Users/arj26323/Documents/Data/Biomass datasets/plotcoord.csv'
# df_fc = geemap.csv_to_pandas(in_csv)

In [11]:
##Water masking

##Load or import the Hansen et al. forest change dataset.
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')

##Select the land/water mask.
datamask = hansenImage.select('datamask')

##Create a binary mask.
watermask = datamask.eq(1)

#watermasked00 = ga00.updateMask(watermask)
#Map.addLayer(watermasked00, vis_param, 'masked')

##Works! Although it is not perfect. Make sure to turn off background layers to see.
##See if there are any other water masks that perform better. Small creeks not masked at all

In [12]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
##Seems to change reflectance values slightly (besides the dividing by 10000 part)
def maskL5sr(image):
  ## Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  ##Get the pixel QA band.
  qa = image.select('pixel_qa')

  ##Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  ##Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

##Map the function over image collection
collection = gaCollectionLS5 \
    .map(maskL5sr)

composite = collection.median() ##what exactly is this doing? median pixel value?

In [13]:
##water-, cloud-, and cloud shadow-masked images - GA LTER :D
gamasked00 = ee.Image(maskL5sr(ga00)).updateMask(watermask) ##Weird salt and pepper noise - though not at any sites, I think
gamasked01 = ee.Image(maskL5sr(ga01)).updateMask(watermask)
gamasked02 = ee.Image(maskL5sr(ga02)).updateMask(watermask)
gamasked03 = ee.Image(maskL5sr(ga03)).updateMask(watermask) ##no data collected in Pennings dataset this year
gamasked04 = ee.Image(maskL5sr(ga04)).updateMask(watermask)
gamasked05 = ee.Image(maskL5sr(ga05)).updateMask(watermask)
gamasked06 = ee.Image(maskL5sr(ga06)).updateMask(watermask)
gamasked07 = ee.Image(maskL5sr(ga07)).updateMask(watermask)
gamasked08 = ee.Image(maskL5sr(ga08)).updateMask(watermask)
gamasked09 = ee.Image(maskL5sr(ga09)).updateMask(watermask)
gamasked10 = ee.Image(maskL5sr(ga10)).updateMask(watermask)
gamasked11 = ee.Image(maskL5sr(ga11)).updateMask(watermask)
gamasked12 = ee.Image(maskL5sr(ga12)).updateMask(watermask)
gamasked13 = ee.Image(maskL5sr(ga13)).updateMask(watermask)
gamasked14 = ee.Image(maskL5sr(ga14)).updateMask(watermask)
gamasked15 = ee.Image(maskL5sr(ga15)).updateMask(watermask)
gamasked16 = ee.Image(maskL5sr(ga16)).updateMask(watermask)
gamasked17 = ee.Image(maskL5sr(ga17)).updateMask(watermask)
gamasked18 = ee.Image(maskL5sr(ga18)).updateMask(watermask)
gamasked19 = ee.Image(maskL5sr(ga19)).updateMask(watermask)

##Masked images with higher tides than desired
gamasked15xxx = ee.Image(maskL5sr(ga15xxx)).updateMask(watermask)
gamasked14xxx = ee.Image(maskL5sr(ga14xxx)).updateMask(watermask)

##Check masked images:
Map.addLayer(gamasked19, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "masked")

##Creating an image collection:
ga_finalcollection = ee.ImageCollection.fromImages([gamasked00, gamasked01, gamasked02, gamasked03, gamasked04, \
                                                    gamasked05, gamasked06, gamasked07, gamasked08, gamasked09, \
                                                    gamasked10, gamasked11, gamasked12, gamasked13, gamasked14, \
                                                    gamasked15, gamasked16, gamasked17, gamasked18, gamasked19])

Map.addLayer(gamasked01, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "masked01")
Map.addLayer(ga_pts)

# ga_finalcollection = ee.ImageCollection.fromImages([gamasked00, gamasked01, gamasked02, gamasked04, \
#                                                     gamasked05, gamasked06, gamasked07, gamasked08, gamasked09, \
#                                                     gamasked10, gamasked11, gamasked12, gamasked13, gamasked14, \
#                                                     gamasked15, gamasked16, gamasked17, gamasked18, gamasked19])

##second one is without 2003, when there were no biomass data collected by Pennings

##Unmasked collection, just to check masked/unmasked pixel values at 30m scale - they match (just /10000)
ga_unmaskedcollection = ee.ImageCollection.fromImages([ga00, ga01, ga02, ga03, ga04, \
                                                    ga05, ga06, ga07, ga08, ga09, \
                                                    ga10, ga11, ga12, ga13, ga14, \
                                                    ga15, ga16, ga17, ga18, ga19])


landsat5_final = ee.ImageCollection.fromImages([gamasked00, gamasked01, gamasked02, gamasked03, gamasked04, \
                                                gamasked05, gamasked06, gamasked07, gamasked08, gamasked09, \
                                                gamasked10, gamasked11])

landsat7_final = ee.ImageCollection.fromImages([gamasked12])

landsat8_final = ee.ImageCollection.fromImages([gamasked13, gamasked14, gamasked15, gamasked16, gamasked17, \
                                                gamasked18, gamasked19])

In [14]:
##TIDAL FILTERING; from Narron et al. 2022
##Utilizes L8 bands 4 and 6 for NDWI, and bands 3 and 4 (for pheno)
##Does it work for Landsat 5?

##Flooding potential = 1 - 1/e^(-1.6+20*ndwi+68.6*pheno) - logistic regression
##Identical formula: 1/e^-(-1.6+20*ndwi+68.6*pheno)

def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B6'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(flats.rename('flats'))

##Notes: This is setup for Landsat 8 - apply to l5? Also, had trouble with np.exp() in equation, so used approx value for e
##np.exp() only takes one argument

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B5'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        })
    
    return image.addBands(flats.rename('flats'))

mergedFLATS = landsat8_final.map(addFLATS)

FLAT19 = addFLATS(gamasked19)
FLAT15x = addFLATS(gamasked15xxx) ##Looking at very likely flooded images
FLAT14x = addFLATS(gamasked14xxx)

# Map.addLayer(mergedFLATS.first(), {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 0.2}, 'Flats')
Map.addLayer(FLAT19, {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 0.2}, 'Flats 2019')
Map.addLayer(FLAT15x, {'bands': ['flats'], 'min': 0, 'max': 1}, 'Flats 2015xxx')
# Map.addLayer(FLAT14x, {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 0.2}, 'Flats 2014xxx')

In [15]:
FLAT19 = addFLATS(gamasked19)
FLAT18 = addFLATS(gamasked18)
FLAT17 = addFLATS(gamasked17)
FLAT16 = addFLATS(gamasked16)
FLAT15 = addFLATS(gamasked15)
FLAT14 = addFLATS(gamasked14)
FLAT13 = addFLATS(gamasked13)
FLAT12 = addFLATS(gamasked12)
FLAT11 = addFLATSL5(gamasked11)
FLAT10 = addFLATSL5(gamasked10)
FLAT09 = addFLATSL5(gamasked09)
FLAT08 = addFLATSL5(gamasked08)
FLAT07 = addFLATSL5(gamasked07)
FLAT06 = addFLATSL5(gamasked06)
FLAT05 = addFLATSL5(gamasked05)
FLAT04 = addFLATSL5(gamasked04)
FLAT02 = addFLATSL5(gamasked02)
FLAT01 = addFLATSL5(gamasked01)
FLAT00 = addFLATSL5(gamasked00)

# flats_param = {'min': -10, 
#              'max': 10, 
#              'bands': ['flats'], 
#              'gamma': 1.5}

Map.addLayer(FLAT19, {'bands': ['flats'], 'min': 0, 'max': 1}, 'Flats 2019')
Map.addLayer(FLAT18, {'bands': ['flats'], 'min': 0, 'max': 1}, 'Flats 2018')

Map

Map(center=[31.434120927658846, -81.34324597505625], controls=(WidgetControl(options=['position', 'transparent…

In [16]:
# #Double checking FLATS equation - all the same! xxx has the 1-1/(1+e^x) formulation, as in the paper
# def addxxx(image):
#     flats = ee.Image(0).expression(
#         '1-1/(1+2.718281828459045**(-1.57 + 20*(RED-NIR)/(RED+NIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
#             'NIR': image.select('B6'),
#             'RED': image.select('B4'),
#             'GREEN': image.select('B3')
#         })
    
#     return image.addBands(flats.rename('flats'))


# def addyyy(image):
#     flats = ee.Image(0).expression(
#         '2.718281828459045**(-1.57 + 20*(RED-NIR)/(RED+NIR) + 68.6*(GREEN-RED)/(GREEN+RED))/(1+2.718281828459045**(-1.57 + 20*(RED-NIR)/(RED+NIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
#             'NIR': image.select('B6'),
#             'RED': image.select('B4'),
#             'GREEN': image.select('B3')
#         })
    
#     return image.addBands(flats.rename('flats'))

# flatxxx = addxxx(gamasked19)
# flatyyy = addyyy(gamasked19)
# Map.addLayer(flatxxx, {'bands': ['flats'], 'min': -1, 'max': 1}, 'xxx')
# Map.addLayer(flatyyy, {'bands': ['flats'], 'min': -1, 'max': 1}, 'yyy')


# ee.Initialize()
# naip = ee.ImageCollection('USDA/NAIP/DOQQ') \
#     .filter(ee.Filter.date('2017-01-01', '2018-12-31'))

# trueColor = naip.select(['R', 'G', 'B'])

# trueColorVis = {
#   min: 0.0,
#   max: 255.0,
# }

# Map.addLayer(trueColor, trueColorVis, 'True Color')


In [17]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first() ##use different NLCD image for earlier years
landcover = nlcd2019.select('landcover')
elev_set = dem2.select('elevation') ##loaded in cell below

landcovermask = landcover.remap([11,31,95], [1,1,1])
elevmask = elev_set.updateMask(elev_set.lte(2))

testy = landcover.updateMask(landcovermask)
testy = testy.updateMask(elevmask)

Map.addLayer(landcover, {}, 'Landcover') ##Look up NLCD landcover key for classes
Map.addLayer(testy, {}, 'test') 

NameError: name 'dem2' is not defined

In [ ]:
##Adding DEM, extracting DEM raster values
import localtileserver
dem = 'E:/Elevation/USGS Sapelo/USGS_1_n32w082.tif'
dem2 = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
dem3 = ee.Image('USGS/SRTMGL1_003') ##1 arc second, or 30 m, and large coverage. Elevation is in meters (from GDEM datum?)

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.add_raster(dem, colormap = "terrain", layer_name='DEM')
Map.addLayer(dem3, dem_params, 'SRTM DEM', True, 1)
Map.addLayer(dem2, dem_params, '10m DEM', True, 1)

##Exporting pixel values for DEM, all plots!
in_fc = fc_all
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'DEM_vals_10m.csv')
# geemap.extract_values_to_points(in_fc, dem2, out_csv)

In [ ]:
##Exporting pixel values from points
coords_year = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/Coords_year/NoFlower/coords_2000.csv'
##NoFlower folder is identical to Coords_year folder, with flowering status removed (confuses csv_to_ee function)
fc_coords = geemap.csv_to_ee(coords_year, latitude = "Latitude", longitude = "Longitude")
# in_fc = fc_coords ##changed from ga_points - getting pixel values for individual plots

Map.addLayer(fc_coords, {}, "fc_coords")

##export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'flats2000.csv')
# geemap.extract_values_to_points(fc_coords, FLAT00, out_csv) ##only works for single images - maybe for loop?

##Previously: geemap.extract_values_to_points(fc_coords, gamasked19, out_csv)

# xyx = ga_finalcollection.toBands()##Does this combine all bands within image collection?
# geemap.extract_values_to_points(fc_all, gamasked19, out_csv) ##compare to 'df' in GA-yearly file

##With 'all points', all points for each year (overlapping) are included

In [ ]:
##Exporting pixel values - zonal statistics: Average 3x3 grid for Landsat \
##(radius = 30 or 90 m - is the 'scale' option in zonal_statistics radius??? ALSO: Be careful with upland veg and water)
##For now, stick with scale = 30m - gets single-pixel information. Will have to think about this for VA though

out_dir = os.path.expanduser('~/Downloads')
out_ga_stats = os.path.join(out_dir, 'ga_avtest.csv')  

# geemap.zonal_statistics(in_value_raster = ga_finalcollection, in_zone_vector = fc, \
#                 out_file_path = out_ga_stats, statistics_type='MEAN', \
#                 scale = 30)

In [18]:
##Streamlining: extracting all the data to a pandas dataframe

##Exporting pixel values from points
ga_flatscollection = ee.ImageCollection.fromImages(
    [
        FLAT00
    ]
)


def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30 
    )
    return feature

xtract = ga_flatscollection.map(addDate).map(rasterExtraction).flatten()

final_extracted = geemap.ee_to_pandas(xtract)

display(final_extracted)


flatty00 = geemap.ee_to_pandas(rasterExtraction(FLAT00)) ##This way is better; imagecollection method runs into data restriction
display(flatty00)

##DO THIS WITH ELEVATION TOO 8/24

out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'ma_extract.csv')
# final_extracted.to_csv(out_csv, index = False)

,Site,Month,Year,Zone,Plot,Latitude,Day,Longitude,B2,B3,B4,B5,B6,B7,flats,imagedate,B1
0,4,10,2000,1,1,31.451676,10,-81.365498,0.0393,0.0421,0.0773,0.0630,0.292,0.0332,0.000368,20001024,0.0315
1,4,10,2000,1,2,31.451548,10,-81.365560,0.0393,0.0383,0.0817,0.0600,0.292,0.0332,0.006054,20001024,0.0292
2,4,10,2000,1,3,31.451473,10,-81.365566,0.0393,0.0383,0.0817,0.0600,0.292,0.0332,0.006054,20001024,0.0292
3,4,10,2000,1,4,31.451373,10,-81.365579,0.0393,0.0383,0.0817,0.0660,0.292,0.0376,0.002479,20001024,0.0292
4,4,10,2000,1,5,31.451197,10,-81.366132,0.0441,0.0458,0.0861,0.0630,0.292,0.0332,0.002402,20001024,0.0338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,5,10,2019,2,4,31.435830,11,-81.341134,0.0439,0.0420,0.0728,0.0569,0.292,0.0332,0.044540,20001024,0.0337
3212,5,10,2019,2,5,31.435748,11,-81.341275,0.0439,0.0420,0.0728,0.0569,0.292,0.0332,0.044540,20001024,0.0337
3213,5,10,2019,2,6,31.435647,11,-81.341406,0.0439,0.0383,0.0772,0.0539,0.292,0.0289,0.430339,20001024,0.0314
3214,5,10,2019,2,7,31.435543,11,-81.341501,0.0439,0.0383,0.0772,0.0539,0.292,0.0289,0.430339,20001024,0.0314


,Site,Month,Year,Zone,Plot,Latitude,Day,Longitude,B2,B3,B4,B5,B6,B7,flats,B1
0,4,10,2000,1,1,31.451676,10,-81.365498,0.0393,0.0421,0.0773,0.0630,0.292,0.0332,0.000368,0.0315
1,4,10,2000,1,2,31.451548,10,-81.365560,0.0393,0.0383,0.0817,0.0600,0.292,0.0332,0.006054,0.0292
2,4,10,2000,1,3,31.451473,10,-81.365566,0.0393,0.0383,0.0817,0.0600,0.292,0.0332,0.006054,0.0292
3,4,10,2000,1,4,31.451373,10,-81.365579,0.0393,0.0383,0.0817,0.0660,0.292,0.0376,0.002479,0.0292
4,4,10,2000,1,5,31.451197,10,-81.366132,0.0441,0.0458,0.0861,0.0630,0.292,0.0332,0.002402,0.0338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,5,10,2019,2,4,31.435830,11,-81.341134,0.0439,0.0420,0.0728,0.0569,0.292,0.0332,0.044540,0.0337
3212,5,10,2019,2,5,31.435748,11,-81.341275,0.0439,0.0420,0.0728,0.0569,0.292,0.0332,0.044540,0.0337
3213,5,10,2019,2,6,31.435647,11,-81.341406,0.0439,0.0383,0.0772,0.0539,0.292,0.0289,0.430339,0.0314
3214,5,10,2019,2,7,31.435543,11,-81.341501,0.0439,0.0383,0.0772,0.0539,0.292,0.0289,0.430339,0.0314


In [34]:
##Streamlining: a time series for each sampling date
ga_2000 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2000, 2000,'year')) \
    .filter(ee.Filter.calendarRange(5, 10,'month')) \
    .filterBounds(ga_pts) \
    .sort('CLOUD_COVER', True)

def maskFLATS(image):
    mask1 = image.select('flats').lte(0.11)
    return image.updateMask(mask1)

ga_2000Flats = ga_2000.map(maskL5sr).map(addFLATSL5).map(maskFLATS)

##Don't even need a water mask!! Flats does that for you


mean00 = ga_2000Flats.mean()

Map.addLayer(mean00, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, 'xxx')

Map 

##DONE!!!!!! Just need to do this for every year

Map(bottom=856147.0, center=[31.385442464173863, -81.29985809326173], controls=(WidgetControl(options=['positi…

In [29]:
##Mean pixel values for growing season at each sampling location, corrected for clouds and flooding!

df00 = geemap.ee_to_pandas(rasterExtraction(mean00))
display(df00)

,Site,Month,Year,Zone,Plot,Latitude,Day,Longitude,B2,B3,B4,B5,B6,B7,flats,B1
0,4,10,2000,1,1,31.451676,10,-81.365498,0.056200,0.056500,0.103800,0.083533,0.295067,0.049000,0.005872,0.043333
1,4,10,2000,1,2,31.451548,10,-81.365560,0.054320,0.051500,0.105180,0.078940,0.296040,0.045080,0.023922,0.040040
2,4,10,2000,1,3,31.451473,10,-81.365566,0.054320,0.051500,0.105180,0.078940,0.296040,0.045080,0.023922,0.040040
3,4,10,2000,1,4,31.451373,10,-81.365579,0.054320,0.051500,0.105180,0.078940,0.296040,0.045080,0.023922,0.040040
4,4,10,2000,1,5,31.451197,10,-81.366132,0.060050,0.057775,0.116775,0.088075,0.294750,0.051000,0.020292,0.046325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,5,10,2019,2,4,31.435830,11,-81.341134,0.061100,0.057540,0.102780,0.079780,0.296820,0.045120,0.065983,0.044120
3212,5,10,2019,2,5,31.435748,11,-81.341275,0.061100,0.057540,0.102780,0.079780,0.296820,0.045120,0.065983,0.044120
3213,5,10,2019,2,6,31.435647,11,-81.341406,0.064267,0.062933,0.108533,0.083267,0.298267,0.049067,0.038570,0.047367
3214,5,10,2019,2,7,31.435543,11,-81.341501,0.064267,0.062933,0.108533,0.083267,0.298267,0.049067,0.038570,0.047367


In [ ]:
##IDEA: instead of for entire growing season, 3 months before? That way we can include MA data (rolling averages for each yr)